### WESAD, CASE

In [14]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
import pandas as pd
import random
import scipy.signal as ss
import shap
import sys
sys.path.append(module_path)

import tools.data_reader_case as dr_c
import tools.data_reader_wesad as dr_w
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

import lightgbm as lgb
from lightgbm import LGBMClassifier
from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)
warnings.simplefilter(action='ignore', category=FutureWarning)

In [15]:
metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    train.Metrics.HF_RR, 
    train.Metrics.LF_RR, 
    train.Metrics.SDNN, 
    train.Metrics.MEAN_SCL, 
    train.Metrics.SCR_RATE,
]
# ] + train.Metrics.STATISTICAL

model_phases_case = dr_c.CLIPS

model_phases_wesad = dr_w.Phases.PHASE_ORDER

case_label_type = dr_c.SelfReports.AROUSAL
wesad_label_type = "stai"

models = {
    "SVM": SVC(),
    "LGB": LGBMClassifier(),
    "RF": RandomForestClassifier(random_state=16),
    "XGB": XGBClassifier(random_state=16),
    # "random": None
}

parameters = {
    "SVM": [{
        "kernel": ["rbf", "poly", "sigmoid"],
        "C": [0.1, 1, 10, 100],
        "gamma": [1, 0.1, 0.01, 0.001, "scale", "auto"],
    }],
    "LGB": [{
        "objective": ["binary"],
        "num_leaves": [10, 20, 30, 40, 50],
        "max_depth": [3, 4, 5, 6, 7],
        "metric": ["binary_logloss"]
    }],
    "RF": [{
        "n_estimators": [10, 20, 30, 40, 50],
        "max_features": ["sqrt", "0.4"],
        "min_samples_split": [3, 4, 5, 6, 7],
        "random_state": [16]
    }],
    "XGB": [{
        "objective": ["binary:logistic"],
        "learning_rate": [0.01, 0.1, 0.3, 0.5],
        "max_depth": [4, 5, 6, 7],
        "n_estimators": [10, 20, 30, 40],
        "eval_metric": ["error"],
        "use_label_encoder": [False],
        "random_state": [16]
    }],
    # "random": None
}

threshold = "fixed"
test_size = 1.0

percent_of_target_dataset = 0.1

temp_a, _ = train.Train_CASE.get_case_data(metrics, verbose=False, label_type=case_label_type, threshold=threshold, normalize=False)
idx = temp_a[temp_a["bpm"] > 200].index 
invalid_case_subjects = set(temp_a["subject"].iloc[idx].tolist())
idx = temp_a[temp_a["bpm"] < 35].index 
invalid_case_subjects.update(set(temp_a["subject"].iloc[idx].tolist()))

temp_a, _ = train.Train_WESAD.get_wesad_data([train.Metrics.BPM], phases=dr_w.Phases.PHASE_ORDER, normalize=False)
idx = temp_a[temp_a["bpm"] > 200].index 
invalid_wesad_subjects = set(temp_a["subject"].iloc[idx].tolist())
idx = temp_a[temp_a["bpm"] < 35].index 
invalid_wesad_subjects.update(set(temp_a["subject"].iloc[idx].tolist()))

In [16]:
# TRAIN ON WESAD AND TEST ON CASE -- ALL
importlib.reload(train)
importlib.reload(dr_c)
importlib.reload(dr_w)
importlib.reload(dt)


random.seed(12)

x_a, y_a = train.Train_WESAD.get_wesad_data(metrics, model_phases_wesad, verbose=False, label_type=wesad_label_type, threshold=threshold, normalize=True)
x_b, y_b = train.Train_CASE.get_case_data(metrics, verbose=False, label_type=case_label_type, threshold=threshold, normalize=True)

x_a = x_a.drop(["phaseId"], axis=1)
x_b = x_b.drop(["phaseId"], axis=1)

inds = pd.isnull(x_a).any(axis=1).to_numpy().nonzero()[0]
x_a = x_a.drop(labels=inds, axis=0).reset_index(drop=True)
y_a = y_a.drop(labels=inds, axis=0).reset_index(drop=True)
inds = pd.isnull(x_b).any(axis=1).to_numpy().nonzero()[0]
x_b = x_b.drop(labels=inds, axis=0).reset_index(drop=True)
y_b = y_b.drop(labels=inds, axis=0).reset_index(drop=True)

# make sure subjects from different datasets aren't labeled with the same index
x_b["subject"] = x_b["subject"] + 500
y_b["subject"] = y_b["subject"] + 500

acc_results = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
reports = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
best_models = {}

num_iters = 1
get_importance = True
for _ in range(num_iters):
    # include a subset of the target dataset in the training data
    subjects = list(np.unique(x_b.loc[:, "subject"]))
    train_subjects = random.sample(subjects, int(len(subjects) * percent_of_target_dataset))
    x_train_addition = x_b[x_b["subject"].isin(train_subjects)]
    y_train_addition = y_b[y_b["subject"].isin(train_subjects)]
    x_train = pd.concat([x_a, x_train_addition])
    y_train = pd.concat([y_a, y_train_addition])
    x_test = x_b[~x_b["subject"].isin(train_subjects)]
    y_test = y_b[~y_b["subject"].isin(train_subjects)]

    # HYPERPARAMETER TUNING
    model_data = train.grid_search_cv(
        # models, parameters, x_a, y_a, by_subject=True, save_metrics=True, is_resample=True,
        models, parameters, x_train, y_train, by_subject=True, save_metrics=True, is_resample=True,
        get_importance=get_importance, drop_subject=True, test_size=0.0, folds=5
    )

    for model_name in models.keys():
        best_models[model_name] = model_data[model_name]["best_model"]
        print(f"{model_name}: {model_data[model_name]['best_params']}")

    # # FEATURE SELECTION
    features = {name: metrics for name in models.keys()}
    # features = train.feature_selection(best_models, model_data["cv"], x_a, y_a, n_features=5)
    # features = train.feature_selection(best_models, model_data["cv"], x_train, y_train, n_features=5)

    # out = train.Train_Multi_Dataset.train_across_datasets(best_models, features, x_a, y_a, x_b, y_b, by_subject=True, save_metrics=True, test_size=test_size, is_resample=False, get_importance=get_importance, drop_subject=True)
    out = train.Train_Multi_Dataset.train_across_datasets(best_models, features, x_train, y_train, x_test, y_test, by_subject=True, save_metrics=True, test_size=test_size, is_resample=False, get_importance=get_importance, drop_subject=True)
    
    for model_name in acc_results:
        acc_results[model_name].append(out[model_name]["performance"][0])
        reports[model_name].append(out[model_name]["performance"][1])
        if get_importance:
            try:
                print("")
                feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name]["performance"][2]))
                feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                print(feature_imp)
            except Exception as e:
                print(out[model_name]["performance"][2])
            print("")

for model_name in acc_results.keys():
    print(f"Model evaluation metrics for {model_name}:")
    for i in range(len(reports[model_name])):
        report = reports[model_name][i]
        acc = acc_results[model_name][i]
        p = report["precision"]
        r = report["recall"]
        f1 = report["f1"]
        auc = report["auc"]
        print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
    print(f"Mean acc: {np.mean([acc_results[model_name][i] for i in range(len(reports[model_name]))])}")
    print(f"Mean F1-score: {np.mean([reports[model_name][i]['f1'] for i in range(len(reports[model_name]))])}")
    print(f"Mean AUC score: {np.mean([reports[model_name][i]['auc'] for i in range(len(reports[model_name]))])}")
    print("\n")

Grid search for SVM ...
Grid search for LGB ...
Grid search for RF ...


One or more of the test scores are non-finite: [0.85027904 0.86221376 0.86670071 0.86366093 0.86001808 0.8589866
 0.86602598 0.86808319 0.86694248 0.8626866  0.86492583 0.87411844
 0.87660647 0.87115129 0.87057143 0.85464956 0.86735935 0.86616792
 0.85931972 0.85303952 0.84517931 0.8692656  0.8633854  0.86397349
 0.86661745        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 10, 'gamma': 1, 'kernel': 'poly'}
LGB: {'max_depth': 4, 'metric': 'binary_logloss', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 5, 'n_estimators': 30, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.5, 'max_depth': 7, 'n_estimators': 40, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Model SVM, Actual: [0 1], [295 110], Predictions: [0 1], [332  73]
coef_ only available for SVC with linear kernel
Model LGB, Actual: [0 1], [295 110], Predictions: [0 1], [345  60]
Model RF, Actual: [0 1], [295 110], Predictions: [0 1], [370  35]
Model XGB, Actual: [0 1], [295 110], Predictions: [0 1], [322  83]

None


[('bpm', 221), ('mean_SCL', 160), ('rmssd', 127), ('hf_rr', 97), ('sdnn', 76), ('SCR_rate', 62), ('lf_rr', 48)]


[('bpm', 0.33105091714916957), ('rmssd', 0.17137050120318972), ('sdnn', 0.11811638376887416), ('SCR_rate', 0.10742051634091837), ('hf_rr', 0.1

In [17]:
# TRAIN ON CASE AND TEST ON WESAD -- ALL
importlib.reload(train)
importlib.reload(dr_c)
importlib.reload(dr_w)
importlib.reload(dt)


random.seed(12)

x_a, y_a = train.Train_CASE.get_case_data(metrics, verbose=False, label_type=case_label_type, threshold=threshold, normalize=True)
x_b, y_b = train.Train_WESAD.get_wesad_data(metrics, model_phases_wesad, verbose=False, label_type=wesad_label_type, threshold=threshold, normalize=True)

x_a = x_a.drop(["phaseId"], axis=1)
x_b = x_b.drop(["phaseId"], axis=1)

inds = pd.isnull(x_a).any(axis=1).to_numpy().nonzero()[0]
x_a = x_a.drop(labels=inds, axis=0).reset_index(drop=True)
y_a = y_a.drop(labels=inds, axis=0).reset_index(drop=True)
inds = pd.isnull(x_b).any(axis=1).to_numpy().nonzero()[0]
x_b = x_b.drop(labels=inds, axis=0).reset_index(drop=True)
y_b = y_b.drop(labels=inds, axis=0).reset_index(drop=True)

# make sure subjects from different datasets aren't labeled with the same index
x_b["subject"] = x_b["subject"] + 500
y_b["subject"] = y_b["subject"] + 500

acc_results = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
reports = {
    "SVM": [],
    "LGB": [],
    "RF": [],
    "XGB": [],
    # "random": []
}
best_models = {}

num_iters = 1
get_importance = True
for _ in range(num_iters):
    # include a subset of the target dataset in the training data
    subjects = list(np.unique(x_b.loc[:, "subject"]))
    train_subjects = random.sample(subjects, int(len(subjects) * percent_of_target_dataset))
    x_train_addition = x_b[x_b["subject"].isin(train_subjects)]
    y_train_addition = y_b[y_b["subject"].isin(train_subjects)]
    x_train = pd.concat([x_a, x_train_addition])
    y_train = pd.concat([y_a, y_train_addition])
    x_test = x_b[~x_b["subject"].isin(train_subjects)]
    y_test = y_b[~y_b["subject"].isin(train_subjects)]

    # HYPERPARAMETER TUNING
    model_data = train.grid_search_cv(
        # models, parameters, x_a, y_a, by_subject=True, save_metrics=True, is_resample=True,
        models, parameters, x_train, y_train, by_subject=True, save_metrics=True, is_resample=True,
        get_importance=get_importance, drop_subject=True, test_size=0.0, folds=5
    )

    for model_name in models.keys():
        best_models[model_name] = model_data[model_name]["best_model"]
        print(f"{model_name}: {model_data[model_name]['best_params']}")

    # # FEATURE SELECTION
    features = {name: metrics for name in models.keys()}
    # features = train.feature_selection(best_models, model_data["cv"], x_a, y_a, n_features=5)
    # features = train.feature_selection(best_models, model_data["cv"], x_train, y_train, n_features=5)

    # out = train.Train_Multi_Dataset.train_across_datasets(best_models, features, x_a, y_a, x_b, y_b, by_subject=True, save_metrics=True, test_size=test_size, is_resample=False, get_importance=get_importance, drop_subject=True)
    out = train.Train_Multi_Dataset.train_across_datasets(best_models, features, x_train, y_train, x_test, y_test, by_subject=True, save_metrics=True, test_size=test_size, is_resample=False, get_importance=get_importance, drop_subject=True)
    
    for model_name in acc_results:
        acc_results[model_name].append(out[model_name]["performance"][0])
        reports[model_name].append(out[model_name]["performance"][1])
        if get_importance:
            try:
                print("")
                feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name]["performance"][2]))
                feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                print(feature_imp)
            except Exception as e:
                print(out[model_name]["performance"][2])
            print("")

for model_name in acc_results.keys():
    print(f"Model evaluation metrics for {model_name}:")
    for i in range(len(reports[model_name])):
        report = reports[model_name][i]
        acc = acc_results[model_name][i]
        p = report["precision"]
        r = report["recall"]
        f1 = report["f1"]
        auc = report["auc"]
        print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
    print(f"Mean acc: {np.mean([acc_results[model_name][i] for i in range(len(reports[model_name]))])}")
    print(f"Mean F1-score: {np.mean([reports[model_name][i]['f1'] for i in range(len(reports[model_name]))])}")
    print(f"Mean AUC score: {np.mean([reports[model_name][i]['auc'] for i in range(len(reports[model_name]))])}")
    print("\n")

Grid search for SVM ...
Grid search for LGB ...
Grid search for RF ...


One or more of the test scores are non-finite: [0.64921945 0.66444646 0.66194501 0.65985643 0.66530651 0.64783968
 0.65201359 0.66149772 0.66839633 0.66755681 0.65192187 0.65838202
 0.66712273 0.67326825 0.67748035 0.63746978 0.66284644 0.67583562
 0.67819988 0.67758123 0.6646991  0.66864586 0.67575695 0.67122026
 0.67214627        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan]


Grid search for XGB ...
SVM: {'C': 0.1, 'gamma': 0.1, 'kernel': 'rbf'}
LGB: {'max_depth': 3, 'metric': 'binary_logloss', 'num_leaves': 10, 'objective': 'binary'}
RF: {'max_features': 'sqrt', 'min_samples_split': 6, 'n_estimators': 40, 'random_state': 16}
XGB: {'eval_metric': 'error', 'learning_rate': 0.5, 'max_depth': 4, 'n_estimators': 10, 'objective': 'binary:logistic', 'random_state': 16, 'use_label_encoder': False}
Model SVM, Actual: [0 1], [393  95], Predictions: [0], [488]
coef_ only available for SVC with linear kernel
Model LGB, Actual: [0 1], [393  95], Predictions: [0 1], [265 223]
Model RF, Actual: [0 1], [393  95], Predictions: [0 1], [261 227]
Model XGB, Actual: [0 1], [393  95], Predictions: [0 1], [292 196]

None


[('rmssd', 89), ('hf_rr', 81), ('lf_rr', 80), ('bpm', 75), ('mean_SCL', 74), ('sdnn', 64), ('SCR_rate', 41)]


[('hf_rr', 0.1815884802686105), ('lf_rr', 0.18129027413353152), ('rmssd', 0.14923158370417366), ('sdnn', 0.13751835863967815), ('bpm', 0.133832715414